In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-259086
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-259086


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "ComputeCluster"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(
    vm_size = "STANDARD_D2_V2",
    min_nodes=0,
    max_nodes=4,
)

try:
    cluster_handle = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already exist. Using it.")
except:
    print("Could not find such cluster. Creating one.")
    cluster_handle = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
cluster_handle.wait_for_completion()

Cluster already exist. Using it.


In [3]:
# pip install azureml-sdk[automl,notebooks,explain,train]

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'C':uniform(0.5,2.0),
                             'max_iter': choice(50, 100, 150, 200)})
# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=2,
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=cluster_handle,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=20,
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
hdr.wait_for_completion(show_output=True)

RunId: HD_a2b4df16-8549-4fec-8862-5935120c1568
Web View: https://ml.azure.com/runs/HD_a2b4df16-8549-4fec-8862-5935120c1568?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-259086/workspaces/quick-starts-ws-259086&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-05-13T20:41:28.781736][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-05-13T20:41:29.3305783Z][SCHEDULER][INFO]Scheduling job, id='HD_a2b4df16-8549-4fec-8862-5935120c1568_0' 
[2024-05-13T20:41:29.3988562Z][SCHEDULER][INFO]Scheduling job, id='HD_a2b4df16-8549-4fec-8862-5935120c1568_1' 
[2024-05-13T20:41:29.5086644Z][SCHEDULER][INFO]Scheduling job, id='HD_a2b4df16-8549-4fec-8862-5935120c1568_2' 
[2024-05-13T20:41:29.6671779Z][SCHEDULER][INFO]Scheduling job, id='HD_a2b4df16-8549-4fec-8862-5935120c1568_3' 
[2024-05-13T20:41:29.636525][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_a2b4df16-8549-4fec-8862-5935120c1568',
 'target': 'ComputeCluster',
 'status': 'Completed',
 'startTimeUtc': '2024-05-13T20:41:27.767742Z',
 'endTimeUtc': '2024-05-13T20:54:35.350593Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'fb662f38-ac15-4dc7-afea-fe99a444f30d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_a2b4df16-8549-4fec-8862-5935120c1568_5',
  'score': '0.915629742033384',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_a2b4df16-8549-4fec-8862-5935120c1568_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVa

In [6]:
best_run = hdr.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_a2b4df16-8549-4fec-8862-5935120c1568_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-05-13-20',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [8]:
import joblib

best_run.register_model(model_name="hyperdrive_opt_best_model", model_path=os.path.join("outputs", "model.pkl"))

Model(workspace=Workspace.create(name='quick-starts-ws-259086', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-259086'), name=hyperdrive_opt_best_model, id=hyperdrive_opt_best_model:1, version=1, tags={}, properties={})

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

URL = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=URL)

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cluster_handle,
    enable_onnx_compatible_models=True)

In [13]:
# Submit your automl run

run = exp.submit(automl_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_dbe21107-8f12-4d7f-9b62-90455cf8cba8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_dbe21107-8f12-4d7f-9b62-90455cf8cba8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()
print(best_run)


import joblib

os.mkdir("model_output")
os.chdir("model_outputs")
joblib.dump(fitted_model,"AUTOML.pkl")

In [ ]:
cluster_handle.delete()